# 05. Iteration 기능과 사람 개입(Human-in-the-loop)

## Iteration 기능과 사람 개입(Human-in-the-loop)

iter() 메서드는 에이전트의 실행 과정을 단계별로 반복할 수 있게 해주는 반복자(iterator)를 생성합니다.

중간 과정에서 사용자의 입력을 받아 계속 진행할지 묻는 기능을 제공합니다. 이를 Human-in-the-loop 라고 합니다.

In [1]:
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_teddynote import logging
from langgraph.prebuilt.chat_agent_executor import create_tool_calling_executor

load_dotenv()
logging.langsmith("CH15-Agents")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agents


In [3]:
from langchain.agents import tool, AgentExecutor


@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together."""
    return a + b

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor

tools = [add_function]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the `search_news` tool for searching keyword related news.",
        ),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

llm = ChatOpenAI(model="gpt-4o-mini")

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    max_iterations=10,
    handle_parsing_errors=True,
)

### AgentExecutor의 iter()

이 메서드는 AgentExecutor의 실행 과정을 단계별로 반복할 수 있게 해주는 반복자(iterator)를 생성합니다.

`iter() 는 에이전트가 최종 출력에 도달하기까지 거치는 단계들을 순차적으로 접근할 수 있는 AgentExecutorIterator 객체를 반환합니다.`

In [17]:
question = "114.5 + 121.2 + 34.2 + 110.1 의 계산 결과는?"

for step in agent_executor.iter({"input": question}):
    print("====")
    if output := step.get("intermediate_step"):
        action, value = output[0]
        print(output)
        if action.tool == "add_function":
            # Tool 실행 결과 출력
            print(f"\nTool Name: {action.tool}, 실행 결과: {value}")
        # 사용자에게 계속 진행할지 묻습니다.
        _continue = input("계속 진행하시겠습니다? (y/n)?:\n") or "Y"
        # 사용자가 'y'가 아닌 다른 입력을 하면 반복 중단
        if _continue.lower() != "y":
            break
            
if "output" in step:
     print(step["output"])
        

====
[(ToolAgentAction(tool='add_function', tool_input={'a': 114.5, 'b': 121.2}, log="\nInvoking: `add_function` with `{'a': 114.5, 'b': 121.2}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_IxPmkpAAlmyBWOXdwNELqeiM', 'function': {'arguments': '{"a":114.5,"b":121.2}', 'name': 'add_function'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b'}, id='run-0be8b661-8296-43ae-850f-5159669a75ad', tool_calls=[{'name': 'add_function', 'args': {'a': 114.5, 'b': 121.2}, 'id': 'call_IxPmkpAAlmyBWOXdwNELqeiM', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'add_function', 'args': '{"a":114.5,"b":121.2}', 'id': 'call_IxPmkpAAlmyBWOXdwNELqeiM', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_IxPmkpAAlmyBWOXdwNELqeiM'), 235.7)]

Tool Name: add_function, 실행 결과: 235.7
====
[(ToolAgentAction(tool='add_function', tool_input={'a':